In [ ]:
import scipy
scipy.__version__

In [ ]:
from autumn.core.project import get_project
from matplotlib import pyplot
from autumn.core.plots.utils import REF_DATE
# from autumn.calibration.targets import get_target_series
import pandas as pd
from autumn.core.utils.display import pretty_print
import matplotlib.pyplot as plt
import datetime

In [ ]:
project = get_project("sm_sir", "ho_chi_minh_city")

In [ ]:
# run baseline model
model_0 = project.run_baseline_model(project.param_set.baseline)
derived_df = model_0.get_derived_outputs_df()

In [ ]:
pretty_print(project.param_set.baseline)

In [ ]:
# run scenarios
start_times = [
    sc_params.to_dict()["time"]["start"] for sc_params in project.param_set.scenarios
]
sc_models = project.run_scenario_models(model_0, project.param_set.scenarios, start_times=start_times)
derived_dfs = [m.get_derived_outputs_df() for m in sc_models]

In [ ]:
len(derived_dfs)

In [ ]:
# define scenario names in compliance with policy brief
scenario_names = {
    0: "Hypertransmissible VoC",
    1: "Hypertransmissible, partially immune-escape VoC",
    2: "Hypertransmissible, partially immune-escape, severe VoC", 
    3: "Hypertransmissible, totally immune-escape VoC",
    4: "Hypertransmissible, totally immune-escape, severe VoC"
}

# derived output figure title lookup
title_lookup = {
    "cdr": "Case Detection Rate (in ratio)",
    "hospital_occupancy": "Total actually occupied hospital beds",
    "icu_occupancy": "Total actuall occupied beds for critical patients",
    "incidence": "Daily actual number of COVID-19 cases",
    "infection_deaths": "Daily number of COVID-19-specific fatalities",
    "notifications": "Daily number of registered cases"
}

In [ ]:
def plot_all_outputs_chosen_scenarios(scenarios_to_plot, name_to_save):
    plot_left_date = datetime.date(2021, 5, 25)
    plot_right_date = datetime.date(2023, 1, 1)
    start_date_scenarios = datetime.date(2022, 7, 1)
    
    outputs_to_plot = ("hospital_occupancy", "icu_occupancy")
    
    fig, axes = plt.subplots(1, 2, figsize = (21, 7), sharex="all")
    fig.set_facecolor("w")
    for i_ax, axis in enumerate(axes.reshape(-1)):
        print(i_ax)
        axis.axvline(x=start_date_scenarios, linestyle="--", color="k")
        plt.text(start_date_scenarios, 0.7, ' 2022-06-01', transform=axis.get_xaxis_transform(), fontsize=12)
        # plot baseline before start date of scenarios
        result_df = derived_dfs[0][outputs_to_plot[i_ax]][plot_left_date:start_date_scenarios]
        indices = result_df.index
        axis.plot(indices, result_df)
        for scenario in scenarios_to_plot:
            results_df = derived_dfs[scenario][outputs_to_plot[i_ax]][start_date_scenarios:plot_right_date]
            indices = results_df.index
            axis.plot(indices, results_df, label = scenario_names[scenario], linewidth=2)
        
        axis.tick_params(axis="x", labelrotation=45)
        axis.set_xlim(left=plot_left_date)
        axis.set_title(title_lookup[outputs_to_plot[i_ax]], fontsize=20)
        axis.legend()
    
    fig.suptitle(name_to_save, fontsize=25)
    # fig.savefig(f"./HCMC_scens_{scenarios_to_plot}_all_outputs.png")

In [ ]:
derived_df["notifications"]

In [ ]:
derived_dfs[2]["notifications"]

In [ ]:
plot_all_outputs_chosen_scenarios([0, 1, 2, 3, 4], "Plotting of all scenarios")

In [ ]:
# plot outputs of scenarios without and with the emergence of Omicron VoC, respectively
# plot_all_outputs_chosen_scenarios([0, 2, 4, 6], 
#     "HCMC modelling results for all outputs of scenarios when Omicron does not emerge")
# plot_all_outputs_chosen_scenarios([1, 3, 5, 7], 
#     "HCMC modelling results for all outputs of scenarios when Omicron emerges on 01st Jan 2022")
plot_all_outputs_chosen_scenarios([1, 2], "Scenarios 2 and 3")

In [ ]:
# # note that scenarios are numbered from 0 for baseline, and 1, 2, 3, etc. onwards
# start_date_scenarios = datetime.date(2021, 6, 1)
# end_date_scenarios = datetime.date(2022, 11, 1)

# def print_epi_info(scenario):
#     print("From {} to {}, for {}:".format(start_date_scenarios, end_date_scenarios, scenario_names[scenario]))
#     print("\b The necessary number of hospital beds needed to fulfill the occupancy is {}."
#           .format(round(max(derived_dfs[scenario]["hospital_occupancy"][start_date_scenarios:end_date_scenarios]))))
#     print("\b The necessary number of ICU beds needed to fulfill the occupancy is {}."
#           .format(round(max(derived_dfs[scenario]["icu_occupancy"][start_date_scenarios:end_date_scenarios]))))
#     print("\b The maximum number of incidence cases (including asymptomatic and undetected cases) is {}."
#           .format(round(max(derived_dfs[scenario]["incidence"][start_date_scenarios:end_date_scenarios]))))
#     print("\b The maximum number of daily COVID-19-specific fatalities is {}."
#           .format(round(max(derived_dfs[scenario]["infection_deaths"][start_date_scenarios:end_date_scenarios]))))
#     print("\b The maximum number of daily COVID-19 registered cases is {}."
#           .format(round(max(derived_dfs[scenario]["notifications"][start_date_scenarios:end_date_scenarios]))))
#     print("\b The total number of daily COVID-19 registered cases is {}."
#           .format(round(sum(derived_dfs[scenario]["notifications"][start_date_scenarios:end_date_scenarios]))))

In [ ]:
# for i in range(0, 3):
#     print_epi_info(i)